# MA Crossing Finder

The purpose of this script is finding the asset(s) which price crossed a particular MA value. The WIG20, mWIG40 and sWIG80 are taken as an examples of tickers.

In [1]:
import pandas as pd
from yahoofinancials import YahooFinancials
import numpy as np
import os
from datetime import datetime

Read the tickers from file (MA_Cross_Data/WIG_tickers.txt).

In [2]:
with open("MA_Cross_Data/WIG_tickers.txt", "r") as file:
    three_wigs_tickers = file.read().splitlines()

In [3]:
len(three_wigs_tickers)

140

Get data from today. Make sure the market is already closed.

In [4]:
def get_current_prices_dict(tickers):
    yahoo_financials = YahooFinancials(tickers)
    return yahoo_financials.get_current_price()

In [5]:
today_price_dict = get_current_prices_dict(three_wigs_tickers)

In [6]:
len(today_price_dict)

140

In [7]:
def get_moving_avg_dict(tickers, ma=50):
    ''' At the moment only ma 50 and 200 is supported by yahoo finance '''
    yahoo_financials = YahooFinancials(tickers)
    if ma == 50:
        return yahoo_financials.get_50day_moving_avg()
    elif ma == 200:
        return yahoo_financials.get_200day_moving_avg()
    else:
        print(f'Wrong ma value {ma}. Only 50 and 200 are supported')

In [8]:
ma = 50
today_ma_dict = get_moving_avg_dict(three_wigs_tickers)

First step - check if prices are above or below ma. We will need to have two files per ma: 1. With yeasterday data (save every day after checking the changes) 2. Change log

In [19]:
ma_position_list = []
for ticker, price in  today_price_dict.items():
    ma_value = today_ma_dict[ticker]
    if price > ma_value:
        print(f'For {ticker} the price {price} is above ma {ma_value}')
        ma_position_list.append('above')
    elif price < ma_value:
        print(f'For {ticker} the price {price} is below ma {ma_value}')
        ma_position_list.append('below')
    else:
        print(f'Wow mate {ticker} is just on the line! {price} = {ma_value}')
        ma_position_list.append('equal')

For ALE.WA the price 22.575 is below ma 29.6525
For ACP.WA the price 76.5 is below ma 77.389
For CCC.WA the price 51.54 is below ma 55.5866
For CDR.WA the price 123.06 is below ma 161.9468
For CPS.WA the price 24.34 is below ma 27.4188
For DNP.WA the price 281.1 is below ma 309.11
For JSW.WA the price 69.6 is above ma 66.4794
For KGH.WA the price 139.05 is below ma 166.178
For LTS.WA the price 66.52 is above ma 60.682
For LPP.WA the price 8885 is below ma 10487.3
For MBK.WA the price 285.4 is below ma 351.608
For OPL.WA the price 6.68 is below ma 7.71654
For PEO.WA the price 92.94 is below ma 108.9524
For PCO.WA the price 41.48 is below ma 43.1985
For PGE.WA the price 9.658 is above ma 9.07252
For PGN.WA the price 6.21 is below ma 6.3872
For PKN.WA the price 74 is below ma 75.5464
For PKO.WA the price 31.62 is below ma 38.3712
For PZU.WA the price 30.71 is below ma 32.7624
For SPL.WA the price 258.2 is below ma 293.878
For 11B.WA the price 530 is below ma 537.33
For ALR.WA the price 34

In [20]:
ma_position_list[:3]

['below', 'below', 'below']

Second step - create the data frame with current data.

In [21]:
price_ma_df = pd.DataFrame(columns=['Ticker', 'Price', 'MA', 'Position'])

In [22]:
price_ma_df['Ticker'] = today_price_dict.keys()
price_ma_df['Price'] = today_price_dict.values()
price_ma_df['MA'] = today_ma_dict.values()
price_ma_df['Position'] = ma_position_list

In [23]:
price_ma_df

,Ticker,Price,MA,Position
0,ALE.WA,22.575,29.6525,below
1,ACP.WA,76.500,77.3890,below
2,CCC.WA,51.540,55.5866,below
3,CDR.WA,123.060,161.9468,below
4,CPS.WA,24.340,27.4188,below
...,...,...,...,...
135,VOX.WA,38.500,40.6040,below
136,VRG.WA,3.670,3.7239,below
137,WWL.WA,473.000,472.6600,above
138,WLT.WA,6.930,7.7928,below


Third step - read last session data.

In [14]:
last_price_ma_df = pd.read_csv(f'MA_Cross_Data/WIG_price_ma{ma}_position.csv')
# last_price_ma_df = price_ma_df.copy()
# last_price_ma_df.loc[1, 'Position'] = 'below'

Fourth step - compare the 'Position' columns and return the ticker for which it changed.

In [24]:
mask = (last_price_ma_df['Position'] != price_ma_df['Position'])

In [25]:
mask

0      False
1      False
2      False
3      False
4      False
       ...  
135    False
136     True
137     True
138    False
139     True
Name: Position, Length: 140, dtype: bool

In [26]:
tickers_ser = price_ma_df[mask]['Ticker']

In [27]:
tickers_ser

23     ASB.WA
25     BFT.WA
31     DAT.WA
36     FMF.WA
52     PEP.WA
70     ABS.WA
74     APR.WA
104    MLS.WA
110    PCF.WA
136    VRG.WA
137    WWL.WA
139    ZEP.WA
Name: Ticker, dtype: object

Print the changes of the positions.

In [28]:
for i in tickers_ser.index:
    if (last_price_ma_df.loc[i]['Position'] == 'above') & (price_ma_df.loc[i]['Position'] == 'below'):
        print(price_ma_df.loc[i]['Ticker'], 'LOST the MA')
    elif (last_price_ma_df.loc[i]['Position'] == 'above') & (price_ma_df.loc[i]['Position'] == 'equal'):
        print(price_ma_df.loc[i]['Ticker'], 'DROPPED perfectly on the MA')
    elif (last_price_ma_df.loc[i]['Position'] == 'below') & (price_ma_df.loc[i]['Position'] == 'above'):
        print(price_ma_df.loc[i]['Ticker'], 'BEAT the MA')
    elif (last_price_ma_df.loc[i]['Position'] == 'below') & (price_ma_df.loc[i]['Position'] == 'equal'):
        print(price_ma_df.loc[i]['Ticker'], 'REACHED perfectly the MA')
    elif (last_price_ma_df.loc[i]['Position'] == 'equal') & (price_ma_df.loc[i]['Position'] == 'above'):
        print(price_ma_df.loc[i]['Ticker'], 'BEAT (from prev. eq) the MA')
    elif (last_price_ma_df.loc[i]['Position'] == 'equal') & (price_ma_df.loc[i]['Position'] == 'below'):
        print(price_ma_df.loc[i]['Ticker'], 'LOST (from prev. eq) the MA')

ASB.WA BEAT the MA
BFT.WA LOST the MA
DAT.WA BEAT the MA
FMF.WA LOST the MA
PEP.WA LOST the MA
ABS.WA LOST the MA
APR.WA BEAT the MA
MLS.WA BEAT the MA
PCF.WA LOST the MA
VRG.WA LOST the MA
WWL.WA BEAT the MA
ZEP.WA LOST the MA


Fifth step - Update the change log (if any changes).

In [34]:
change_log_df = pd.read_csv(f'MA_Cross_Data/WIG_price_ma{ma}_change_log.csv')
# change_log_df = pd.DataFrame(columns=['Index', 'Date', 'Ticker', 'Prev. Price', 'Curr. Price', 'Prev. MA', 'Curr. MA', 'Prev. Position', 'Curr. Position'])

In [36]:
today = datetime.today().strftime('%Y-%m-%d')
for idx in tickers_ser.index:
#     print(today, price_ma_df.loc[idx]['Ticker'])
#     print(last_price_ma_df.loc[idx]['Price'], price_ma_df.loc[idx]['Price'])
#     print(last_price_ma_df.loc[idx]['MA'], price_ma_df.loc[idx]['MA'])
#     print(last_price_ma_df.loc[idx]['Position'], price_ma_df.loc[idx]['Position'])
    change_log_df.loc[len(change_log_df)] = [change_log_df.last_valid_index()+1, today, price_ma_df.loc[idx]['Ticker'], 
                                             last_price_ma_df.loc[idx]['Price'], price_ma_df.loc[idx]['Price'],
                                             last_price_ma_df.loc[idx]['MA'], price_ma_df.loc[idx]['MA'], 
                                             last_price_ma_df.loc[idx]['Position'], price_ma_df.loc[idx]['Position']]

In [37]:
change_log_df

,Date,Ticker,Prev. Price,Curr. Price,Prev. MA,Curr. MA,Prev. Position,Curr. Position
0,2022-05-04,ASB.WA,12.41,13.390,12.45680,12.45680,below,above
1,2022-05-04,BFT.WA,602.00,590.000,590.10000,590.10000,above,below
2,2022-05-04,DAT.WA,173.70,176.200,173.98480,173.98480,below,above
3,2022-05-04,FMF.WA,2.80,2.766,2.78808,2.78808,above,below
4,2022-05-04,PEP.WA,68.80,66.500,67.67200,67.67200,above,below
5,2022-05-04,ABS.WA,41.80,41.300,41.56000,41.56000,above,below
6,2022-05-04,APR.WA,14.40,14.860,14.68440,14.68440,below,above
7,2022-05-04,MLS.WA,67.60,70.750,68.53000,68.53000,below,above
8,2022-05-04,PCF.WA,58.50,56.400,57.46920,57.46920,above,below
9,2022-05-04,VRG.WA,3.75,3.670,3.72390,3.72390,above,below


Sixth step - Save both log files.

In [38]:
price_ma_df.to_csv(f'MA_Cross_Data/WIG_price_ma{ma}_position.csv')
change_log_df.to_csv(f'MA_Cross_Data/WIG_price_ma{ma}_change_log.csv')